In [ ]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone
import bcrypt

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

In [ ]:
def user_create_account(user_id, password, user_name, user_description, profile_pic, 
        profile_background, birthday, email, country, language, fund, 
        filtering, notification, cookies):

    encoded_password = password.encode('utf-8')
    hashed_password = bcrypt.hashpw(encoded_password, bcrypt.gensalt())

    cur.execute(
        """
        INSERT INTO public."user" 
        (user_id, password_hashed, user_name, user_description, profile_pic, 
        profile_background, birthday, email, country, language, fund, 
        filtering, notification, cookies) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            user_id, 
            hashed_password, 
            user_name, 
            user_description, 
            profile_pic, 
            profile_background, 
            birthday, 
            email, 
            country, 
            language, 
            fund, 
            filtering, 
            notification, 
            cookies
        )
    )

    conn.commit()
    return

In [ ]:
def user_login(user_id, password):
    cur.execute(
        """
        SELECT password_hashed 
        FROM public."user" 
        WHERE user_id = %s
        """,
        (user_id,)
    )
    conn.commit()
    stored_password_hash = cur.fetchone()[0] 
    encoded_password = password.encode('utf-8')

    success = False
    if bcrypt.checkpw(encoded_password, stored_password_hash):
        success = True

    return success

In [ ]:
def user_change_settings():

    return

In [ ]:
def user_add_fund(user_id, amount):
    cur.execute(
        """
        UPDATE public."user" 
        SET fund = fund + %s
        WHERE user_id = %s FOR UPDATE;
        """,
        (amount, user_id,)
    )
    
    conn.commit()
    return

In [ ]:
def user_search_for_games(keywords):
    cur.execute(
        """
        SELECT password_hashed 
        FROM public."game" 
        WHERE game_name LIKE %s 
        """,
        (f"%{keywords}%",)
    )
    conn.commit()
    related_games = cur.fetchall()
    
    # related_games is a list
    return related_games

In [ ]:
def user_buy_items():
    # add transaction
    
    # update fund

    

    return

In [ ]:
def user_uninstall_games():

    return

In [ ]:
def user_add_friends(user_id, friend_id):
    cur.execute(
        """
        INSERT INTO public."user_friends" 
        (user_id, friend_id) 
        VALUES (%s, %s)
        """,
        (user_id, friend_id)
    )
    cur.execute(
        """
        INSERT INTO public."user_friends" 
        (user_id, friend_id) 
        VALUES (%s, %s)
        """,
        (friend_id, user_id)
    )

    conn.commit()
    return

In [ ]:
def user_add_shopping_cart(user_id, game_id, item_id):
    success = False
    try:
        cur.execute(
            """
            INSERT INTO public."cart" 
            (user_id, game_id, item_id) 
            VALUES (%s, %s, %s)
            ON CONFLICT (user_id, game_id, item_id) DO NOTHING
            """,
            (user_id, game_id, item_id)
        )

        conn.commit()
        success = True
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.commit().rollback()
        
    return success

In [ ]:
def user_delete_shopping_cart(user_id, game_id, item_id):
    cur.execute(
        """
        DELETE FROM public."cart"
        WHERE user_id = %s AND game_id = %s AND item_id = %s
        """,
        (user_id, game_id, item_id)
    )

    conn.commit() 
    return

In [ ]:
def user_review_games(review_id, user_id, game_id, rating, text, review_timestamp):
    cur.execute(
        """
        INSERT INTO public."game_reviews" 
        (review_id, user_id, game_id, rating, text, review_timestamp) 
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        (review_id, user_id, game_id, rating, text, review_timestamp)
    )

    conn.commit()
    return

In [ ]:
def user_look_up_for_achievements():

    return